In [3]:
#Sample dataset
#Format: each row is an example
# The last column is the label
#The first two columns are the features
#2nd and 5th e.g have same features but different labels

training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [9]:
#Column labels
#These are used only to print the tree
header = ["color", "diameter", "label"]

def unique_vals(rows, col):
    '''Find the unique values for a column in a dataset'''
    return set([row[col] for row in rows])

########
# DEMO:
# unique_vals(training_data, 0)
# unique_vals(training_data, 1)
########


def class_counts(rows):
    '''Counts the number of each type of examples in a dataset'''
    counts = {} #a dictionary of Label -> count
    for row in rows:
        #in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

########
# DEMO:
# class_counts(training_data)
########


def is_numeric(value):
    '''Test if a value is numeric'''
    return isinstance(value, int) or isinstance(value, float)

########
# DEMO:
# is_numeric(7)
# is_numeric("Red")
########


class Question:
    '''A question is used to partition a data a dataset.
    This class just records a 'column number' (e.g., 0 for color) and
    the feature value in an example to the feature value stored in the 
    question. See the demo below.
    '''

    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self,example):
        #compare the feature value in an example to the
        #feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        #This is just a helper method to print
        #The question in a readable format
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))
    

def partition(rows, question):
        '''Partitions a dataset.
        For each row in the dataset, check if it matches the question. If 
        so, add it to the 'true rows', otherwise, add it to 'false rows.' 
        '''
        
        true_rows, false_rows = [], []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows
    
    
########
# DEMO:
# Let's partition the training data based on whether rows are Red.
# true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# This will contain all the 'Red' rows.
# true_rows
#This will contain everything else.
# false_rows
########



def gini(rows):
    '''Calculate the gini impurity for a list of rows.'''
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    '''Information Gain
    The uncertanity of the starting node, minus the weighted impurity of
    two child nodes.
    '''
    
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

########
# DEMO:
# Calculate the uncertainty of our training data.
# current_uncertainty = gini(training_data)
# 
# How much information do we gain by partioning on 'Green'?
# true_rows, false_rows = partition(training_data, Question(0, 'Green'))
# info_gain(true_rows, false_rows, current_uncertainty)
#
# What about if we partioned on 'Red' instead?
# true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# info_gain(true_rows, false_rows, current_uncertainty)
########

def find_best_split(rows):
    '''Find the best question to ask by iterating over every feature/ value
    and calculating the information gain.
    '''
    best_gain = 0     #keep track of the best information gain
    best_question = None      #keep train of the feature/ value that produced
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1      #number of columns
    
    for col in range(n_features):      #for each feature
        values = set([row[col] for row in rows])   #unique values in the col
        
        for val in values:   #for each value
            question = Question(col,val)
            
            #try splitting the dataset
            true_rows, false_rows = partition(rows,question)
            #Skip this split if it dosen't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            #Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            #You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our toy dataset
            if gain >= best_gain:
                best_gain, best_question= gain, question
                
    return best_gain, best_question

########
# DEMO:
# Find the best question to ask first for our toy dataset
# best_gain, best_question = find_best_split(training_data)
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='
########

class Leaf:
    '''A leaf node classifies data
    This holds a dictionary of class(e.g. Mango) -> number of times
    it appears in the rows from the training data that reach this leaf'''
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
class Decision_Node:
    '''A Decision Node aska a question
    This holds a referance to the question and to the two child nodes'''
    
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
def build_tree(rows):
    '''Builds the tree'''
    # Try partitioning the dataset on each of the unique attribute
    # calculate the information gain
    # and return the question that produces the highest gain
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain
    # Since we can ask no further questions
    # We'll return a leaf
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature/value to partition valye
    true_rows, false_rows = partition(rows,question)
    
    #Recursively build the true branch
    true_branch = build_tree(true_rows)
    
    #Recursively build the false branch
    false_branch = build_tree(false_rows)
    
    # Return a question node
    # This records the best feature/value to aske at this point, as well 
    # as the branches to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, spacing=""):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    #Print the questions at this node
    print(spacing + str(node.question))
    
    #Call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    
    #Call this function recursively on the false branch
    print(spacing + '-->False:')
    print_tree(node.false_branch, spacing +" ")
    
def classify(row, node):
    #Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    #Decide whether to follow the true-branch or the false-branch
    #Compare the future/value stored in the node, to the eg we're considering
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
    
###########
# DEMO:
# The tree predicts the first row of our training data is an apple
# with confidence 1.
# my_tree = build_tree(training_data)
# classify(training_data[0], my_tree)
###########

def print_leaf(counts):
    '''Print the predictions at a leaf'''
    total = sum(counts.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total*100)) + "%"
    return probs
    

###########
# DEMO:
# Printing that a bit nicer
# print_leaf(classify(training_data[0], my_tree)) 
###########

###########
# DEMO:
# On the second example, the confidence is lower
# print_leaf(classify(training_data[1], my_tree)) 
###########

if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    
    # Evaluate
    testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 4, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]
    
    for row in testing_data:
        print("Actual: %s. Predicted: %s" %
              (row[-1], print_leaf(classify(row, my_tree)))) 
        

# Next steps
# -add support for missing (or unseen) attributes
# -prune the tree to prevent overfitting
# -add support for regression
    

Is diameter >= 3?
--> True:
 Is color == Yellow?
 --> True:
  Predict {'Mango': 1, 'Lemon': 1}
 -->False:
  Predict {'Mango': 1}
-->False:
 Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
